In [1]:
source_dir = '/Users/mesoor/data/resumes/智联/text/2021/01/'

files = list(list_files(source_dir, pattern='*.txt'))
len(files)

31744

In [2]:
def preprocess(text):
    return text.strip().lower().replace('：', ':').replace('（', '(').replace('）', ')')

docs = {}
for file in files:
    lines = [preprocess(line) for line in read_lines(file) if line.strip()]
    if lines:
        docs[file] = lines
        
len(docs)

31111

In [3]:
text2docs = {'\n'.join(v): k for k, v in docs.items()}
docs = {v: k for k, v in text2docs.items()}
len(docs)

27921

In [4]:
first(docs.values())

'龙希 求职意向:律师/法务实习生\n出生年月:1999 年 8 月 性 别:女\n籍 贯:湖南长沙 毕业学校:上海立信会计金融学院\n学 历:大学本科 电 话:18573109731\n工作年限:应届毕业生 邮 箱:longxi18573109731@163.com\n教育背景\n2017.9-至今 上海立信会计金融学院 经济法学/本科\n主修课程: 法理学，宪法学，刑法，民法，刑事诉讼法，民事诉讼法，行政法与行政诉讼法，国际\n法，国际私法，国际经济法，国际税法环境法与资源保护法，物权法，合同法，公司法，商法，知识产权\n法，经济法，金融法学，税法，会计学原理，金融学原理，财务会计，数据库应用等。\n实习经历\n2019.11-2020.1 上海市浦东新区法院自贸区法庭 书记员和法官助理\n工作描述: 1. 整理案卷材料，旁听庭审；\n2. 独立完成自立案起到开庭审理前的法律文书拟写与送达工作;\n2018.7-2018.8 湖南省常德市桃源县法院 书记员和法官助理\n工作描述: 1. 整理案卷材料，拟写法律文书，旁听庭审与部分工作会议；\n2. 帮助当事人办理取保候审相关材料；\n3. 协助被告律师查阅相关案卷材料。\n个人技能\n专业技能: 已通过 2020 年司法考试客观题\n语言能力: 通过大学英语 cet6、普通话二级甲等\n办公技能: 通过全国计算机等级考试(二级 c)，熟练掌握 word、excel、ppt 等办公软件\n学校荣誉: 在校期间多次多次获得奖学金一等奖，二等奖及三好学生等\n自我评价\n沟通能力很强，性格开朗。有责任感和较高的工作热情，处事冷静，做事稳重积极向上。并能吃苦耐劳，\n具有优秀的团队合作精神。做事稳重，善于沟通，有一定的谈判能力及很强的责任心。在以后的职业生涯\n中，将一直以务实、稳重、责任心强的工作态度来对待岗位职责。'

In [5]:
import codecs

import jieba
import nltk
from sklearn import feature_extraction

In [6]:
# stopwords = nltk.corpus.stopwords.words('english')
# len(stopwords)

In [7]:
def tokenize_only(text):
    if isinstance(text, list):
        text = '\n'.join(text)
    tokens = [word for word in jieba.cut(text) if word.strip()]
    return tokens

def tokenize_and_stem(text):
    return tokenize_only(text)

In [8]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for doc in docs.values():
    allwords_stemmed = tokenize_and_stem(doc) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(doc)
    totalvocab_tokenized.extend(allwords_tokenized)
    
len(totalvocab_stemmed), len(totalvocab_tokenized)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/fs/2r8gpw9s1bb2pngp9rsty5mr0000gn/T/jieba.cache
Loading model cost 0.824 seconds.
Prefix dict has been built successfully.


(19524732, 19524732)

In [9]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 19524732 items in vocab_frame


In [10]:
len(set(vocab_frame.words))

387444

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.9,
                                   max_features=200000,
                                   min_df=50,
                                   use_idf=True,
                                   tokenizer=tokenize_only,
                                   ngram_range=(1,3))
tfidf_vectorizer

TfidfVectorizer(max_df=0.9, max_features=200000, min_df=50, ngram_range=(1, 3),
                tokenizer=<function tokenize_only at 0x7fd3bbf9f830>)

In [12]:
docs_ids, docs_texts = zip(*docs.items())
%time tfidf_matrix = tfidf_vectorizer.fit_transform(docs_texts)
type(tfidf_matrix), tfidf_matrix.shape

CPU times: user 5min 29s, sys: 7.4 s, total: 5min 37s
Wall time: 5min 40s


(scipy.sparse.csr.csr_matrix, (27921, 66427))

In [13]:
terms = tfidf_vectorizer.get_feature_names()
len(terms)

66427

In [14]:
terms[:10]

['!',
 '! 此致',
 '! 此致 敬礼',
 '"',
 '" ----',
 '" ---- =',
 '" ;',
 '" ; type',
 '" mailto',
 '" mailto :']

In [16]:
# from sklearn.metrics.pairwise import cosine_similarity

# dist = 1 - cosine_similarity(tfidf_matrix)
# type(dist), dist.shape

# KMeans

In [17]:
from sklearn.cluster import KMeans

n_clusters = 100
km = KMeans(n_clusters=n_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
len(clusters) # save as docs length

CPU times: user 31min 19s, sys: 3min 1s, total: 34min 20s
Wall time: 19min 38s


27921

In [18]:
#from sklearn.externals import joblib
# deprecated in 0.21, removed in 0.23
import joblib

joblib.dump(km, 'doc_cluster_100.pkl')
clusters = km.labels_.tolist()

In [19]:
docs_clusters = {'files': docs_ids, 'docs': docs_texts, 'cluster': clusters}
df_docs = pd.DataFrame(docs_clusters)
len(df_docs), df_docs.columns

(27921, Index(['files', 'docs', 'cluster'], dtype='object'))

In [20]:
df_docs['cluster'].value_counts()

13    1845
1     1184
99     990
43     628
54     554
      ... 
52      45
39      42
0       42
77      37
10      14
Name: cluster, Length: 100, dtype: int64

## Top terms per cluster

In [21]:
# km.cluster_centers_: ndarray of shape (n_clusters, n_features), coordinates of cluster centers
# 得到每个 centroid 的 tfidf 值最高的词
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(n_clusters):
    print(f'Cluster {i} words:')
    words = []
    for ind in order_centroids[i, :100]:
        words.append(terms[ind])
    print(words)

Cluster 0 words:
['\uf074', '；', ')', '项目', '与', '等', '和', '及', '熟悉', '(', '能力', '进行', '管理', '负责', '技术', '食品', '客户', '分析', '活动', '完成', '会计', '变电站', '产品', '、 食品', '并', '对', '在', '企业', '系统', '相关', '优秀', '计算机', '证书', '工程', '一', '招标', '年', '强', '了', '组织', '供电', '我', '二级', '团队', '软件', '沟通', '\uf0fe', '业务', '各', '数据', '哈尔滨', '情况', '( 本科 )', '公司', '( 本科', '本科 )', '有限公司', '奖项', '专业', '开发', '费用', '岁', '个性 开朗', '“', '学习', '奖项 证书', ';', '财务', '确定', '知识', '工艺', '中', '设计', '为', '2019', '良好', '有', '经验', '教育', '良好 的', '操作', '2017.9', '以及', '文件', '国网', '大赛', '部门', '背景', '基础', '申报', '—', '教育 背景', '技能', ',', '容易 相处 ，', '个性', '建设', '荣誉感 强', '开朗', '经营']
Cluster 1 words:
['·', '。 ·', '· 负责', '和', '与', '等', '；', '年', ')', '(', '活动', '能力', '负责', '项目', '/', '“', '及', '； ·', '”', '管理', '了', '并', ') ·', '在', '进行', '月', '获得', '—', '组织', '证书 ·', '有', '学院', '客户', '公司', '证书', '奖学金', '学习', '经历', '实习', '软件', '2019', '~', '2018', '技能', '大学', '团队', '对', '强', '设计', '专业', ': ·', '实践', '相关', '英语', '学生', '参加', '有限公司', '课程'

## 分离 docs

In [22]:
import shutil

def copy_file(src, dst):
    if not os.path.exists(os.path.dirname(dst)):
        os.makedirs(os.path.dirname(dst))
    shutil.copy2(src, dst)

In [24]:
grouped = df_docs.groupby(by=['cluster'])
for key, sub_df in grouped:
    print(key)
    # print(sub_df.files.iloc[0])
    for text_file in sub_df.files:
        src_file = text_file.replace('/text/', '/files/')[:-4]
        if os.path.exists(src_file):
            dst_file = src_file.replace('/files/', f'/files100/{key}/')
            copy_file(src_file, dst_file)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


# 查看词频分布

In [35]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(tokenizer=tokenize_only)
X = count_vectorizer.fit_transform(docs.values())
X.shape

(8596, 199197)

In [39]:
sums = sum(X.toarray())
sums.shape

(199197,)

In [44]:
df_sums = pd.DataFrame.from_records([{'count': n} for n in sums])
df_sums.head()

,count
0,439
1,892
2,926
3,6
4,2


In [50]:
c_sums = Counter(df_sums['count'])
len(c_sums)

1571

In [65]:
sum(cnt for k, cnt in c_sums.most_common() if k >= 50)

7780

In [54]:
c_sums.most_common()

[(1, 119582),
 (2, 24117),
 (3, 10430),
 (4, 8526),
 (5, 4291),
 (6, 3286),
 (7, 2612),
 (8, 2025),
 (9, 1618),
 (10, 1367),
 (11, 1254),
 (12, 1038),
 (13, 927),
 (14, 792),
 (15, 755),
 (16, 634),
 (17, 616),
 (18, 536),
 (19, 489),
 (20, 467),
 (21, 404),
 (23, 398),
 (22, 352),
 (25, 334),
 (24, 325),
 (27, 297),
 (28, 284),
 (26, 277),
 (30, 252),
 (29, 236),
 (31, 206),
 (33, 204),
 (32, 202),
 (36, 201),
 (35, 184),
 (34, 175),
 (37, 162),
 (40, 158),
 (39, 155),
 (38, 154),
 (41, 146),
 (42, 137),
 (46, 132),
 (43, 130),
 (44, 122),
 (47, 117),
 (45, 111),
 (48, 107),
 (56, 102),
 (50, 99),
 (53, 96),
 (54, 94),
 (49, 93),
 (57, 91),
 (51, 90),
 (52, 88),
 (55, 86),
 (63, 80),
 (58, 77),
 (60, 76),
 (65, 74),
 (67, 73),
 (64, 68),
 (59, 68),
 (71, 66),
 (66, 65),
 (61, 64),
 (68, 62),
 (85, 60),
 (62, 60),
 (70, 56),
 (72, 55),
 (84, 54),
 (74, 54),
 (80, 54),
 (76, 52),
 (78, 52),
 (69, 51),
 (73, 49),
 (89, 49),
 (86, 46),
 (79, 45),
 (81, 45),
 (97, 45),
 (75, 44),
 (83, 44)